In [94]:
import pandas as pd
import random, time
import os

# filename = 'titanic.csv'
# filename = 'kaggle_train.csv'
filename = 'kaggle_test.csv'

data = pd.read_csv(os.path.join('..','..','data',filename))

# Understanding the data

In [95]:
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [96]:
data.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [97]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


# Embarked

As we can see there are only 2 missing values in the Embarked column. As a result, the 2 missing Embarked values are filled using the most common value (mode).

In [98]:
data[data['Embarked'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [99]:
x = data['Embarked'].mode()[0]
data['Embarked'].fillna(x, inplace=True)

# Age

The following scatter plot shows that higher values of 'SibSp' have a smaller 'Age' range. For instance it is more likely that a person with 4 other siblings on board the Titanic was a child with their family than an adult. As a reuslt, the missing values of age are computed by generating a random number within the range of known ages of the 'SibSp' value for that particlar person. 

In [100]:
p = data.plot(kind = 'scatter', x = 'Age', y = 'SibSp')

In [101]:
data['SibSp'].value_counts()

0    283
1    110
2     14
3      4
4      4
8      2
5      1
Name: SibSp, dtype: int64

The maximum and minimum ages for each value of 'SibSp' is found and the missing 'Age' values in the training and testing data are computed.

In [102]:
ageRanges = {0: [100, 0], 1: [100,0], 2: [100,0], 3: [100, 0], 4: [100, 0], 5: [100, 0], 8: [100, 0] }
for x in data.index:
    if data.loc[x, 'Age'] < ageRanges[data.loc[x, "SibSp"]][0]:
        ageRanges[data.loc[x, "SibSp"]][0] = data.loc[x, "Age"]
    elif data.loc[x, 'Age'] > ageRanges[data.loc[x, "SibSp"]][1]:
        ageRanges[data.loc[x, "SibSp"]][1] = data.loc[x, "Age"]

ageRanges[8] = [data['Age'].min(), ageRanges[5][1]] 

for x in data.index:
    if pd.isna(data.loc[x, 'Age']):
        random.seed(time.perf_counter())

        data.loc[x, 'Age'] = random.randint(int(ageRanges[data.loc[x, 'SibSp']][0]), int(ageRanges[data.loc[x, 'SibSp']][1]))

# Cabin

Extracting the first character of the cabin value since this corresponds to the deck where the cabin was located and adding it as a new column in the dataframe.

In [103]:
data['Deck'] = data['Cabin'].apply(lambda d: d[0] if pd.notnull(d) else None)

In [104]:
print(data['Deck'].value_counts())

C    35
B    18
D    13
E     9
F     8
A     7
G     1
Name: Deck, dtype: int64


Replacing each letter with a corresponding number

In [105]:
data['Deck'] = data['Deck'].replace(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'], [0, 1, 2, 3, 4, 5, 6, 7])

The correlation between Deck and Pclass is about 0.6. The missing 'Deck' values will be computed similar to how the 'Age' values were worked out. For instance if a person's Pclass is 3, the Deck will be a random number between 4 and 6 as seen in the scatter plot below. For a Pclass value of 1 the random number will be between 0 and 4 because the 7 (which represents the T value before these were changed to numbers) only features once in the dataset and was not included on purpose.

In [106]:
corr=data.corr()
corr.style.background_gradient(cmap='coolwarm')

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Deck
PassengerId,1.000000,-0.026751,0.006708,0.003818,0.043080,0.008211,0.047787
Pclass,-0.026751,1.000000,-0.303071,0.001087,0.018721,-0.577147,0.643199
Age,0.006708,-0.303071,1.000000,-0.116063,0.003319,0.233965,-0.301461
SibSp,0.003818,0.001087,-0.116063,1.000000,0.306895,0.171539,-0.105774
Parch,0.043080,0.018721,0.003319,0.306895,1.000000,0.230046,-0.023149
Fare,0.008211,-0.577147,0.233965,0.171539,0.230046,1.000000,-0.296106
Deck,0.047787,0.643199,-0.301461,-0.105774,-0.023149,-0.296106,1.000000


In [107]:
p2 = data.plot(kind = 'scatter', x = 'Deck', y = 'Pclass')

Setting the appropriate deck ranges for each class and generating the missing values randomly.

In [108]:
# The 7 is not included in Pclass 1 because it only featured once in the whole dataset
deckRanges = {1: [0, 4], 2: [3, 5], 3: [4, 6]}

for x in data.index:
    if pd.isna(data.loc[x, 'Deck']):
        random.seed(time.perf_counter())

        data.loc[x, 'Deck'] = random.randint(deckRanges[data.loc[x, 'Pclass']][0], deckRanges[data.loc[x, 'Pclass']][1])

# Name, Ticket and Cabin

The column 'Cabin is no longer needed since the 'Deck' column was computed from it. The columns 'Name' and 'Ticket' are also dropped from the dataframe.

In [109]:
data.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

# Fare

In [110]:
data[data['Fare'].isnull()]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck
152,1044,3,male,60.5,0,0,NaN,S,6.0


There is only one missing 'Fare' value and this is found in the testing data. The median Fare of all the male passengers in Pclass 3 is used to fill the missing value. 

In [111]:
m = data.groupby(['Pclass', 'Sex']).Fare.median()[3][1]
data.Fare.fillna(m, inplace=True)


# Non-Numerical Values

The categorical values 'Pclass' and 'Embarked' are converted into dummy indicator variables. 

In [112]:
data = pd.get_dummies(data,columns=['Pclass', 'Embarked', 'Deck'])

In [113]:
data

,PassengerId,Sex,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Deck_0.0,Deck_1.0,Deck_2.0,Deck_3.0,Deck_4.0,Deck_5.0,Deck_6.0
0,892,male,34.5,0,0,7.8292,0,0,1,0,1,0,0,0,0,0,1,0,0
1,893,female,47.0,1,0,7.0000,0,0,1,0,0,1,0,0,0,0,0,1,0
2,894,male,62.0,0,0,9.6875,0,1,0,0,1,0,0,0,0,0,1,0,0
3,895,male,27.0,0,0,8.6625,0,0,1,0,0,1,0,0,0,0,0,0,1
4,896,female,22.0,1,1,12.2875,0,0,1,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,male,58.0,0,0,8.0500,0,0,1,0,0,1,0,0,0,0,0,1,0
414,1306,female,39.0,0,0,108.9000,1,0,0,1,0,0,0,0,1,0,0,0,0
415,1307,male,38.5,0,0,7.2500,0,0,1,0,0,1,0,0,0,0,1,0,0
416,1308,male,48.0,0,0,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,1


Since n-1 columns are needed, 'Pclass_3', 'Embarked_S' and 'Deck_7' are dropped.

In [114]:
data.drop(['Pclass_3','Embarked_S'], axis=1, inplace=True)

try: data.drop(['Deck_7.0'], axis=1, inplace=True)
except: print('Skipping Deck')

KeyError: "['Deck_7.0'] not found in axis"

In [ ]:
data.head()

Since there are only 2 values for Categorical column 'Sex', these values are mapped to zero or one. 

In [ ]:
sex_values = {"male":0, "female":1}
data['Sex'] = data["Sex"].map(sex_values)

In [ ]:
data.head()

# Saving the dataframes

In [ ]:
# data.to_csv(os.path.join('..','..',"data",'Variant 3',"titanic_cleaned.csv"),index=False)
# data.to_csv(os.path.join('..','..',"data",'Variant 3',"kaggle_test.csv"),index=False)
data.to_csv(os.path.join('..','..',"data",'Variant 3',"kaggle_train.csv"),index=False)